In [ ]:
# APPL, MSFT 주식 수익률 비교
from pandas_datareader import data as pdr
import yfinance as yf
import matplotlib.pyplot as plt
yf.pdr_override()

aapl = pdr.get_data_yahoo('aapl',start="2018-05-04", progress = False)
aapl_dpc = (aapl["Close"]-aapl['Close'].shift(1)) / aapl['Close'].shift(1) * 100
aapl_dpc.iloc[0] = 0 # 일간 변동률의 첫 번째 값인 NaN을 0으로 변경한다.
aapl_dpc_cs = aapl_dpc.cumsum()

msft = pdr.get_data_yahoo('MSFT',start="2018-05-04", progress = False)
msft_dpc = (msft["Close"]-msft['Close'].shift(1)) / msft['Close'].shift(1) * 100
msft_dpc.iloc[0] = 0
msft_dpc_cs = msft_dpc.cumsum()

plt.plot(aapl.index, aapl_dpc_cs, 'b', label="APPLE``")
plt.plot(msft.index, msft_dpc_cs, 'r--', label="Microsoft")
plt.ylabel("change %")
plt.grid(True)
plt.legend(loc='best')
plt.show()

In [ ]:
# Max Draw Down 최대 손실 낙폭
from pandas_datareader import data as pdr
import yfinance as yf
import matplotlib.pyplot as plt
yf.pdr_override()


# 1984년 12월 01일부터 AAPL데이터를 불러온다.
aapl = pdr.get_data_yahoo("aapl", "1984-12-01", progress = False)

window = 252 # 어림잡아 1년동안 개장일
peak = aapl['Adj Close'].rolling(window, min_periods=1).max()
drawdown = aapl['Adj Close']/peak - 1.0
max_dd = drawdown.rolling(window, min_periods=1).min()

plt.figure(figsize=(9,7))
plt.subplot(211) # 2행 1열 중 1행에 그린다.
aapl["Close"].plot(label="AAPL",title="AAPL MDD", grid=True, legend=True)
plt.subplot(212) # 2행 1열중 2행에 그린다.
drawdown.plot(c='blue', label="AAPL DD", grid=True, legend=True)
max_dd.plot(c="red", label="AAPL MDD", grid=True, legend=True)
plt.show()

# 최대 낙폭은 약 80% 정도로 볼 수 있다.
max_dd[max_dd == max_dd.min()]

In [ ]:
import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yf
from scipy import stats
import matplotlib.pylab as plt
yf.pdr_override()

AAPL = pdr.get_data_yahoo("AAPL","2000-01-04", progress = False)
MSFT = pdr.get_data_yahoo("MSFT","2000-01-04", progress = False)

df = pd.DataFrame({"X": AAPL["Close"], "Y": MSFT["Close"]})
df = df.fillna(method="bfill")
df = df.fillna(method="ffill")

regr = stats.linregress(df.X, df.Y)
regr_line = f'Y = {regr.slope:.2f} * X + {regr.intercept:.2f}'

plt.figure(figsize=(7,7))
plt.plot(df.X, df.Y, ".")
plt.plot(df.X, regr.slope * df.X + regr.intercept, 'r')
plt.legend(["AAPL x MSFT", regr_line])
plt.title(f"AAPL x MSFT (R = {regr.rvalue:.2f})")
plt.xlabel("AAPL")
plt.ylabel("MSFT")
plt.show()

# AAPL과 MSFT의 상관계수가 0.98로 매우 높은 것을 볼 수 있다.
df.corr()

In [ ]:
# AAPL 캔들차트 그리기
from pandas_datareader import data as pdr
import yfinance as yf
import mplfinance as mpf
yf.pdr_override()

aapl = pdr.get_data_yahoo('aapl',start="2022-09-01", progress = False)
mpf.plot(aapl, title='APPLE candle chart', type='candle')
mpf.plot(aapl, title='APPLE ohlc chart', type='ohlc')
kwargs = dict(title='APPLE customized chart', type='candle', mav=(2, 4, 6), volume=True, ylabel='ohlc candles')
mc = mpf.make_marketcolors(up='r', down='b', inherit=True)
s  = mpf.make_mpf_style(marketcolors=mc)
mpf.plot(aapl, **kwargs, style=s)

In [ ]:
# Efficient Frontier 효율적 투자선
# 주어진 리스크에서 최대 수익을 낸다.
# 포트폴리오 리스크 = (종목별 비중)T((종목별 연간 공분산)(종목별 비중))^1/2
from pandas_datareader import data as pdr
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
df = pd.DataFrame()
yf.pdr_override()
stocks = ["AAPL", "GOOGL", "TSLA", "NVDA"]
for i in stocks:
    df[i] = pdr.get_data_yahoo(i, start="2020-01-01", progress = False)['Close']

daily_ret = df.pct_change() # 일간 변동률
annual_ret = daily_ret.mean() * 252 # 연간 수익률
daily_cov = daily_ret.cov() # 일간 리스크
annual_cov = daily_cov * 252 # 연간 리스크

port_ret = []
port_risk = []
port_weights = []

for _ in range(20000):
    weights = np.random.random(len(stocks))
    weights /= np.sum(weights)

    returns = np.dot(weights, annual_ret)
    risk = np.sqrt(np.dot(weights.T, np.dot(annual_cov, weights)))

    port_ret.append(returns)
    port_risk.append(risk)
    port_weights.append(weights)

portfolio = {'Returns': port_ret, 'Risk': port_risk}
for i, s in enumerate(stocks):
    portfolio[s] = [weights[i] for weight in port_weights]
df = pd.DataFrame(portfolio)
df = df[['Returns', 'Risk'] + [s for s in stocks]]

df.plot.scatter(x='Risk', y='Returns', figsize=(8,6), grid=True)
plt.title('Efficient Frontier')
plt.xlabel('Risk')
plt.ylabel('Expected Returns')
plt.show()

In [ ]:
# Portfolio Optimization 포트폴리오 최적화
# 샤프지수 = (포트폴리오 예상 수익률 - 무위험률) / 수익률의 표준편차
from pandas_datareader import data as pdr
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
df = pd.DataFrame()
yf.pdr_override()
stocks = ["AAPL", "GOOGL", "TSLA", "NVDA"]
for i in stocks:
    df[i] = pdr.get_data_yahoo(i, start="2018-01-01", progress = False)['Close']

daily_ret = df.pct_change() # 일간 변동률
annual_ret = daily_ret.mean() * 252 # 연간 수익률
daily_cov = daily_ret.cov() # 일간 리스크
annual_cov = daily_cov * 252 # 연간 리스크

port_ret = []
port_risk = []
port_weights = []
sharpe_ratio = []

for _ in range(2000000):
    weights = np.random.random(len(stocks))
    weights /= np.sum(weights)

    returns = np.dot(weights, annual_ret)
    risk = np.sqrt(np.dot(weights.T, np.dot(annual_cov, weights)))

    port_ret.append(returns)
    port_risk.append(risk)
    port_weights.append(weights)
    sharpe_ratio.append(returns/risk)

portfolio = {'Returns': port_ret, 'Risk': port_risk, 'Sharpe': sharpe_ratio}
for i, s in enumerate(stocks):
    portfolio[s] = [weights[i] for weight in port_weights]
df = pd.DataFrame(portfolio)
df = df[['Returns', 'Risk', 'Sharpe'] + [s for s in stocks]]

max_sharp = df.loc[df['Sharpe'] == df['Sharpe'].max()]
min_risk = df.loc[df['Risk'] == df['Risk'].min()]

df.plot.scatter(x='Risk', y='Returns', c="Sharpe", cmap='viridis', edgecolors='k', figsize=(11,7), grid=True)
plt.scatter(x=max_sharp['Risk'], y=max_sharp['Returns'], c='r', marker="*", s = 300)
plt.scatter(x=min_risk['Risk'], y=min_risk['Returns'], c='r', marker="X", s = 200)
plt.title('Portfolio Optimization')
plt.xlabel('Risk')
plt.ylabel('Expected Returns')
plt.show()

max_sharp

In [ ]:
# Bollinger Bands
# 상단 볼린저 밴드 = 중간 볼린저 밴드 + (2 * 표준편차)
# 중간 볼린저 밴드 = 종가의 20일 이동평균
# 하단 볼린저 밴드 = 중간 볼린저 밴드 - (2 * 표준편차)
from pandas_datareader import data as pdr
import yfinance as yf
import matplotlib.pyplot as plt
yf.pdr_override()

df = pdr.get_data_yahoo("AAPL","2022-02-04", progress = False)
df['MA20'] = df["Close"].rolling(window=20).mean() # 20개 종가를 이용해서 평균을 구한다
df["stddev"] = df["Close"].rolling(window=20).std() # 20개의 종가로 표준편차를 구한다
df["upper"] = df["MA20"] + (df["stddev"] * 2) # 상단 볼린저 밴드
df["lower"] = df["MA20"] - (df["stddev"] * 2) # 하단 볼린저 밴드
df = df[19:] # 19행까지 NaN이기 때문에 20행부터 사용한다

plt.figure(figsize=(9,5))
plt.plot(df.index, df['Close'], color="#0000ff", label="Close")
plt.plot(df.index, df['upper'], "r--", label="Upper band")
plt.plot(df.index, df['MA20'], "k--", label="Moving average 20")
plt.plot(df.index, df['lower'], "c--", label="Lower band")
plt.fill_between(df.index, df["upper"], df["lower"], color="0.9")
plt.legend(loc="best")
plt.title("APPLE Bollinger Band (20 day, 2 std)")
plt.show()

In [ ]:
# Bollinger Bands %b
# %b = (종가 - 하단 볼린저 밴드) / (상단 볼린저 밴드 - 하단 볼린저 밴드), 주가가 볼린저 밴드 어디에 위치하는지를 나타내는 지표
from pandas_datareader import data as pdr
import yfinance as yf
import matplotlib.pyplot as plt
yf.pdr_override()

df = pdr.get_data_yahoo("AAPL","2022-02-04", progress = False)
df['MA20'] = df["Close"].rolling(window=20).mean() # 20개 종가를 이용해서 평균을 구한다
df["stddev"] = df["Close"].rolling(window=20).std() # 20개의 종가로 표준편차를 구한다
df["upper"] = df["MA20"] + (df["stddev"] * 2) # 상단 볼린저 밴드
df["lower"] = df["MA20"] - (df["stddev"] * 2) # 하단 볼린저 밴드
df["PB"] = (df['Close'] - df["lower"]) / (df["upper"] - df["lower"]) # %b 칼럼을 생성한다
df = df[19:] # 19행까지 NaN이기 때문에 20행부터 사용한다

plt.figure(figsize=(9,8))
plt.subplot(2,1,1) # 기존 볼린저 밴드 차트를 2행 1열의 그리드에서 1열에 배치한다
plt.plot(df.index, df['Close'], color="#0000ff", label="Close")
plt.plot(df.index, df['upper'], "r--", label="Upper band")
plt.plot(df.index, df['MA20'], "k--", label="Moving average 20")
plt.plot(df.index, df['lower'], "c--", label="Lower band")
plt.fill_between(df.index, df["upper"], df["lower"], color="0.9")
plt.legend(loc="best")
plt.title("APPLE Bollinger Band (20 day, 2 std)")

plt.subplot(2,1,2) # %b 차트를 2행 1열의 그리드에서 1열에 배치한다
plt.plot(df.index, df["PB"], color="b", label="%B")
plt.grid(True)
plt.legend(loc="best")
plt.show()

In [ ]:
# Bollinger Bands BandWidth
# 상단 볼린저 밴드와 하단 볼린저 밴드 사이의 폭을 의미한다.
# 밴드폭이 6개월 저점을 기록하는 것을 보고 스퀴즈를 파악할 수 있다고 한다. 앞으로 변동성이 심해질 예정
from pandas_datareader import data as pdr
import yfinance as yf
import matplotlib.pyplot as plt
yf.pdr_override()

df = pdr.get_data_yahoo("AAPL","2022-02-04", progress = False)
df['MA20'] = df["Close"].rolling(window=20).mean() # 20개 종가를 이용해서 평균을 구한다
df["stddev"] = df["Close"].rolling(window=20).std() # 20개의 종가로 표준편차를 구한다
df["upper"] = df["MA20"] + (df["stddev"] * 2) # 상단 볼린저 밴드
df["lower"] = df["MA20"] - (df["stddev"] * 2) # 하단 볼린저 밴드
df["bandwidth"] = (df["upper"] - df["lower"]) / df["MA20"] * 100
df = df[19:] # 19행까지 NaN이기 때문에 20행부터 사용한다

plt.figure(figsize=(9,8))
plt.subplot(2,1,1) # 기존 볼린저 밴드 차트를 2행 1열의 그리드에서 1열에 배치한다
plt.plot(df.index, df['Close'], color="#0000ff", label="Close")
plt.plot(df.index, df['upper'], "r--", label="Upper band")
plt.plot(df.index, df['MA20'], "k--", label="Moving average 20")
plt.plot(df.index, df['lower'], "c--", label="Lower band")
plt.fill_between(df.index, df["upper"], df["lower"], color="0.9")
plt.legend(loc="best")
plt.title("APPLE Bollinger Band (20 day, 2 std)")

plt.subplot(2,1,2) # %b 차트를 2행 1열의 그리드에서 1열에 배치한다
plt.plot(df.index, df["bandwidth"], color="m", label="BandWidth")
plt.grid(True)
plt.legend(loc="best")
plt.show()

In [ ]:
# MFI(현금흐름지표, Money Flow Index)
# Typical Price(중심 가격) = 일정 기간의 고가, 저가, 종가를 합한 뒤에 3으로 나눈 값
# 일반적으로 주가에 선행함, 거래량 데이터에 RSI개념을 도입한 지표로 생각할 수 있다.
# MFI = 100 - (100/(1+PMF/NMF))
from pandas_datareader import data as pdr
import yfinance as yf
import matplotlib.pyplot as plt
yf.pdr_override()

df = pdr.get_data_yahoo("AAPL","2022-02-04", progress = False)
df['MA20'] = df["Close"].rolling(window=20).mean() # 20개 종가를 이용해서 평균을 구한다
df["stddev"] = df["Close"].rolling(window=20).std() # 20개의 종가로 표준편차를 구한다
df["upper"] = df["MA20"] + (df["stddev"] * 2) # 상단 볼린저 밴드
df["lower"] = df["MA20"] - (df["stddev"] * 2) # 하단 볼린저 밴드
df["PB"] = (df['Close'] - df["lower"]) / (df["upper"] - df["lower"]) # %b 칼럼을 생성한다
df["TP"] = (df["High"] + df["Low"] + df["Close"]) / 3 # Typical price를 구한다
df["PMF"] = 0
df["NMF"] = 0
for i in range(len(df.Close) - 1):
    if df.TP.values[i] < df.TP.values[i+1]:
        # i번째 중심 가격보다 i+1번째 중심가격이 높면 i+1번째 중심가격과 i+1번째 거래량의 곱을 pmf(positive money flow)에 저장한다
        df.PMF.values[i+1] = df.TP.values[i+1] * df.Volume.values[i+1]
        df.NMF.values[i+1] = 0
    else:
        df.NMF.values[i+1] = df.TP.values[i+1] * df.Volume.values[i+1]
        df.PMF.values[i+1] = 0
df["MFR"] = df.PMF.rolling(window=10).sum() / df.NMF.rolling(window=10).sum()
df["MFI10"] = 100 - 100/ (1+df["MFR"])
df = df[19:] # 19행까지 NaN이기 때문에 20행부터 사용한다

plt.figure(figsize=(9,8))
plt.subplot(2,1,1) # 기존 볼린저 밴드 차트를 2행 1열의 그리드에서 1열에 배치한다
plt.plot(df.index, df['Close'], color="#0000ff", label="Close")
plt.plot(df.index, df['upper'], "r--", label="Upper band")
plt.plot(df.index, df['MA20'], "k--", label="Moving average 20")
plt.plot(df.index, df['lower'], "c--", label="Lower band")
plt.fill_between(df.index, df["upper"], df["lower"], color="0.9")
plt.title("APPLE Bollinger Band (20 day, 2 std) - Trend Following")
for i in range(len(df.Close)):
    if df.PB.values[i] > 0.8 and df.MFI10.values[i] > 80:
        # %b가 0.8보다 크고 10일 기준 MFI가 80보다 매수시점을 나타내기 위해 그래프의 종가 위치에 빨간색 삼각형을 그린다.
        plt.plot(df.index.values[i], df.Close.values[i], "r^")
    elif df.PB.values[i] < 0.2 and df.MFI10.values[i] < 20:
        plt.plot(df.index.values[i], df.Close.values[i], "bv")
plt.legend(loc="best")

plt.subplot(2,1,2) # %b 차트를 2행 1열의 그리드에서 1열에 배치한다
plt.plot(df.index, df["PB"] * 100, "b", label="%B x 100")
plt.plot(df.index, df["MFI10"], "g--", label="MFI(10 day)")
plt.yticks([-20, 0, 20, 40, 60, 80, 100, 120])
for i in range(len(df.Close)):
    if df.PB.values[i] > 0.8 and df.MFI10.values[i] > 80:
        plt.plot(df.index.values[i], 0, "r^")
    elif df.PB.values[i] < 0.2 and df.MFI10.values[i] < 20:
        plt.plot(df.index.values[i], 0, "bv")
plt.grid(True)
plt.legend(loc="best")
plt.show()

In [ ]:
# Intraday intensity(II) 일중 강도
# II = (2 * 종가 - 고가 - 저가) / (고가 - 저가) * 거래량 -> 종가 고가와 저가에 비해 상대적으로 어느정도에 위치하는지를 나타냄
# II% = II(21일 합)/거래량(21일 합) * 100
# 존 볼린저는 일중 강도율을 볼린저 밴드를 확증하는 도구로 사용함, 하단 볼린저, 일중 강도율 + -> 매수
from pandas_datareader import data as pdr
import yfinance as yf
import matplotlib.pyplot as plt
yf.pdr_override()

df = pdr.get_data_yahoo("AAPL","2022-02-04", progress = False)
df['MA20'] = df["Close"].rolling(window=20).mean() # 20개 종가를 이용해서 평균을 구한다
df["stddev"] = df["Close"].rolling(window=20).std() # 20개의 종가로 표준편차를 구한다
df["upper"] = df["MA20"] + (df["stddev"] * 2) # 상단 볼린저 밴드
df["lower"] = df["MA20"] - (df["stddev"] * 2) # 하단 볼린저 밴드
df["PB"] = (df['Close'] - df["lower"]) / (df["upper"] - df["lower"]) # %b 칼럼을 생성한다

# Intraday intensity
df["II"] = (2*df["Close"] - df["High"] - df["Low"]) / (df["High"] - df["Low"]) * df["Volume"]
# II%
df["IIP21"] = df["II"].rolling(window=21).sum() / df["Volume"].rolling(window=21).sum() * 100
df.dropna() # NaN value 제거하기

plt.figure(figsize=(9,9))
plt.subplot(3,1,1) # 기존 볼린저 밴드 차트를 3행 1열 그리드에서 1열에 배치한다
plt.title("APPLE Bollinger Band (20 day, 2 std) - Reversals")
plt.plot(df.index, df['Close'], color="#0000ff", label="Close")
plt.plot(df.index, df['upper'], "r--", label="Upper band")
plt.plot(df.index, df['MA20'], "k--", label="Moving average 20")
plt.plot(df.index, df['lower'], "c--", label="Lower band")
plt.fill_between(df.index, df["upper"], df["lower"], color="0.9")
for i in range(0, len(df.Close)):
    if df.PB.values[i] < 0.05 and df.IIP21.values[i] > 0:
        plt.plot(df.index.values[i], df.Close.values[i], "r^")
    elif df.PB.values[i] > 0.95 and df.IIP21.values[i] < 0:
        plt.plot(df.index.values[i], df.Close.values[i], "bv")
plt.legend(loc="best")

plt.subplot(3,1,2) # %b 차트를 3행 1열의 2그리드 열에 배치한다
plt.plot(df.index, df["PB"], "b", label="%B")
plt.grid(True)
plt.legend(loc="best")

plt.subplot(3,1,3) # IIP21% 차트를 3행 1열의 3그리드 열에 배치한다.
plt.bar(df.index, df["IIP21"], color="g", label="II% 21day")
for i in range(0, len(df.Close)):
    if df.PB.values[i] < 0.05 and df.IIP21.values[i] > 0:
        plt.plot(df.index.values[i], 0, "r^")
    elif df.PB.values[i] > 0.95 and df.IIP21.values[i] < 0:
        plt.plot(df.index.values[i], 0, "bv")
plt.grid(True)
plt.legend(loc="best")
plt.show()

In [ ]:
# 삼중창 매매 시스탬 - 첫 번째 창(추세 확인)
from pandas_datareader import data as pdr
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from mplfinance.original_flavor import candlestick_ohlc
yf.pdr_override()

df = pdr.get_data_yahoo("AAPL","2022-02-04", progress = False)
ema60 = df.Close.ewm(span=60).mean() # 종가의 12주 지수 이동평균
ema130 = df.Close.ewm(span=130).mean() # 종가의 26주 지수 이동평균
macd = ema60 - ema130 # macd선
signal = macd.ewm(span=45).mean() # 신호선(macd의 9주 지수 이동평균선)
macdhist = macd - signal # macd 히스토그램

df = df.assign(ema60=ema60, ema130=ema130, macd=macd, signal=signal, macdhist=macdhist).dropna()
df["Number"] = df.index.map(mdates.date2num)
ohlc = df[["Number", "Open", "High", "Low", "Close"]]

plt.figure(figsize=(9,7))
p1 = plt.subplot(2, 1, 1)
plt.title("Triple Screen Trading - First Screen (APPLE)")
plt.grid(True)
candlestick_ohlc(p1, ohlc.values, width=.6, colorup="red", colordown="blue") # 차트 생성
p1.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
plt.plot(df.Number, df["ema130"], color="c", label="EMA130")
plt.legend(loc="best")

p2 = plt.subplot(2, 1, 2)
p2.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
plt.grid(True)
plt.bar(df.Number, df["macdhist"], color="m", label="MACD-Hist")
plt.plot(df.Number, df["macd"], color="b", label="MACD")
plt.plot(df.Number, df["signal"], "g--", label="MACD-Signal")
plt.legend(loc="best")
plt.show()

In [ ]:
# 삼중창 매매 시스탬 - 두 번째 창(시장 파동)
# 오실레이터 -> 시장이 하락할 때 매수 기회, 시장이 상승할 때 매도 기회
from pandas_datareader import data as pdr
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from mplfinance.original_flavor import candlestick_ohlc
yf.pdr_override()

df = pdr.get_data_yahoo("AAPL","2022-02-04", progress = False)
ema60 = df.Close.ewm(span=60).mean() # 종가의 12주 지수 이동평균
ema130 = df.Close.ewm(span=130).mean() # 종가의 26주 지수 이동평균
macd = ema60 - ema130 # macd선
signal = macd.ewm(span=45).mean() # 신호선(macd의 9주 지수 이동평균선)
macdhist = macd - signal # macd 히스토그램

df = df.assign(ema60=ema60, ema130=ema130, macd=macd, signal=signal, macdhist=macdhist).dropna()
df["Number"] = df.index.map(mdates.date2num)
ohlc = df[["Number", "Open", "High", "Low", "Close"]]

ndays_high = df.High.rolling(window=14, min_periods=1).max() # 14일 동안의 최댓값을 구한다
ndays_low = df.Low.rolling(window=14, min_periods=1).min() # 14일 동안의 최솟값을 구한다.
fast_k = (df.Close - ndays_low) / (ndays_high - ndays_low) * 100 # %k를 구한다.
slow_d = fast_k.rolling(window=3).mean()
df = df.assign(fast_k=fast_k, slow_d=slow_d).dropna()

plt.figure(figsize=(9,7))
p1 = plt.subplot(2, 1, 1)
plt.title("Triple Screen Trading - Second Screen (APPLE)")
plt.grid(True)
candlestick_ohlc(p1, ohlc.values, width=.6, colorup="red", colordown="blue") # 차트 생성
p1.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
plt.plot(df.Number, df["ema130"], color="c", label="EMA130")
plt.legend(loc="best")

p2 = plt.subplot(2, 1, 2)
p2.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
plt.grid(True)
plt.plot(df.Number, df["fast_k"], color="c", label="%K")
plt.plot(df.Number, df["slow_d"], color="k", label="%D")
plt.yticks([0,30,70,100])
plt.legend(loc="best")
plt.show()
# ex) 130일 지수 이동 평균이 상승하고 있을 때 스토캐스틱이 30 아래로 내려가면 매수 기회, 하락하고 있을 때 70위로 올라가면 매도 기회

In [ ]:
# 삼중창 매매 시스탬 - 세 번째 창(진입 기술)
from pandas_datareader import data as pdr
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from mplfinance.original_flavor import candlestick_ohlc
yf.pdr_override()

df = pdr.get_data_yahoo("AAPL","2021-02-04", progress = False)
ema60 = df.Close.ewm(span=60).mean() # 종가의 12주 지수 이동평균
ema130 = df.Close.ewm(span=130).mean() # 종가의 26주 지수 이동평균
macd = ema60 - ema130 # macd선
signal = macd.ewm(span=45).mean() # 신호선(macd의 9주 지수 이동평균선)
macdhist = macd - signal # macd 히스토그램

df = df.assign(ema60=ema60, ema130=ema130, macd=macd, signal=signal, macdhist=macdhist).dropna()
df["Number"] = df.index.map(mdates.date2num)
ohlc = df[["Number", "Open", "High", "Low", "Close"]]

ndays_high = df.High.rolling(window=14, min_periods=1).max() # 14일 동안의 최댓값을 구한다
ndays_low = df.Low.rolling(window=14, min_periods=1).min() # 14일 동안의 최솟값을 구한다.
fast_k = (df.Close - ndays_low) / (ndays_high - ndays_low) * 100 # %k를 구한다.
slow_d = fast_k.rolling(window=3).mean()
df = df.assign(fast_k=fast_k, slow_d=slow_d).dropna()

plt.figure(figsize=(9,9))
p1 = plt.subplot(3, 1, 1)
plt.title("Triple Screen Trading (APPLE)")
plt.grid(True)
candlestick_ohlc(p1, ohlc.values, width=.6, colorup="red", colordown="blue") # 차트 생성
p1.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
plt.plot(df.Number, df["ema130"], color="c", label="EMA130")
for i in range(1,len(df.Close)):
    if df.ema130.values[i-1] < df.ema130.values[i] and \
            df.slow_d.values[i-1] >= 30 and df.slow_d.values[i] < 30:
        plt.plot(df.Number.values[i], 120, "r^")
    elif df.ema130.values[i-1] < df.ema130.values[i] and \
            df.slow_d.values[i-1] <= 30 and df.slow_d.values[i] > 30:
        plt.plot(df.Number.values[i], 120, "bv")
plt.legend(loc="best")

p2 = plt.subplot(3, 1, 2)
p2.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
plt.grid(True)
plt.bar(df.Number, df["macdhist"], color="m", label="MACD-Hist")
plt.plot(df.Number, df["macd"], color="b", label="MACD")
plt.plot(df.Number, df["signal"], "g--", label="MACD-Signal")
plt.legend(loc="best")


p3 = plt.subplot(3, 1, 3)
p3.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
plt.grid(True)
plt.plot(df.Number, df["fast_k"], color="c", label="%K")
plt.plot(df.Number, df["slow_d"], color="k", label="%D")
plt.yticks([0,30,70,100])
plt.legend(loc="best")
plt.show()

In [193]:
# Backtrader - RSI
# RS = N일간의 상승폭의 평균 / N일간의 하락폭의 평균
# RSI = 100 - 100 / (1 + RS), 가격의 움직임의 강도를 백분율로 나타내며 추세가 언제 전환될지 예측하는데 유용함
import backtrader as bt
import yfinance as yf
%matplotlib inline

class MyStrategy(bt.Strategy):
    def __init__(self):
        self.rsi = bt.indicators.RSI(self.data.close)
    def next(self):
        if not self.position:
            if self.rsi < 30:
                self.order = self.buy()
        else:
            if self.rsi > 70:
                self.order = self.sell()

cerebro = bt.Cerebro()
cerebro.addstrategy(MyStrategy)
data = bt.feeds.PandasData(dataname = yf.download('AAPL', '2018-02-04', "2022-12-18", progress = False))
cerebro.adddata(data)
cerebro.broker.setcash(100000)
cerebro.addsizer(bt.sizers.SizerFix, stake=30)

print(f"Initial Portfolio Value : {cerebro.broker.get_value():,.0f} USD")
cerebro.run()
print(f"Final Portfolio Value   : {cerebro.broker.get_value():,.0f} USD")
cerebro.plot()

Initial Portfolio Value : 100,000 USD
Final Portfolio Value   : 101,380 USD


<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 5 Axes>]]

In [199]:
# Backtrader - RSI_SMA
# RSI_SMA : RSA의 21일 단순 이동 평균
import backtrader as bt
import yfinance as yf
%matplotlib inline

class MyStrategy(bt.Strategy):
    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.rsi = bt.indicators.RSI_SMA(self.data.close, period=21)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY  : price {order.executed.price:,.2f}, '
                    f'size {order.executed.size:,.0f}, '
                    f'commission {order.executed.comm:,.2f}, '
                    f'total {cerebro.broker.getvalue():,.2f}')
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:
                self.log(f'SELL : price {order.executed.price:,.2f}, '
                    f'size {order.executed.size:,.0f}, '
                    f'commission {order.executed.comm:,.2f}, '
                    f'total {cerebro.broker.getvalue():,.2f}')
            self.bar_executed = len(self)
        elif order.status in [order.Canceled]:
            self.log('ORDER CANCELD')
        elif order.status in [order.Margin]:
            self.log('ORDER MARGIN')
        elif order.status in [order.Rejected]:
            self.log('ORDER REJECTED')
        self.order = None

    def next(self):
        if not self.position:
            if self.rsi < 30:
                self.order = self.buy()
        else:
            if self.rsi > 70:
                self.order = self.sell()

    def log(self, txt, dt=None):
        dt = self.datas[0].datetime.date(0)
        print(f'[{dt.isoformat()}] {txt}')

cerebro = bt.Cerebro()
cerebro.addstrategy(MyStrategy)

data = bt.feeds.PandasData(dataname = yf.download('AAPL', '2018-02-04', "2022-12-18", progress = False))

cerebro.adddata(data)
cerebro.broker.setcash(100000)
cerebro.broker.setcommission(commission=0.0014)
cerebro.addsizer(bt.sizers.PercentSizer, percents=90)

print(f'Initial Portfolio Value : {cerebro.broker.getvalue():,.0f} USD')
cerebro.run()
print(f'Final Portfolio Value   : {cerebro.broker.getvalue():,.0f} USD')
cerebro.plot(style='candlestick')

Initial Portfolio Value : 100,000 USD
[2018-11-21] BUY  : price 44.93, size 2,034, commission 127.96, total 98,371.87
[2019-02-05] SELL : price 43.22, size -2,034, commission 123.07, total 96,255.36
[2019-06-03] BUY  : price 43.90, size 1,979, commission 121.65, total 94,995.60
[2019-06-25] SELL : price 49.61, size -1,979, commission 137.46, total 107,293.20
[2020-03-02] BUY  : price 70.57, size 1,413, commission 139.60, total 112,992.79
[2020-05-06] SELL : price 75.11, size -1,413, commission 148.59, total 113,427.06
[2021-02-25] BUY  : price 124.68, size 814, commission 142.15, total 110,279.79
[2021-04-20] SELL : price 135.02, size -814, commission 153.94, total 121,551.80
[2021-10-05] BUY  : price 139.49, size 786, commission 153.54, total 122,671.96
[2021-11-03] SELL : price 150.39, size -786, commission 165.54, total 129,802.67
[2022-01-27] BUY  : price 162.45, size 732, commission 166.38, total 127,273.37
[2022-07-20] SELL : price 151.12, size -732, commission 154.77, total 121,

<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 5 Axes>]]